# Kuramoto Music for Life Perceives

Jan 2023, Jonathan Reus


In [ ]:
import asyncio
import time
import traceback
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.animation as animation
import random
import IPython
from scipy import integrate
import oscpipe.oscpipe as oscpipe

# Integration using Euler's method
def euler_integrate(dXfunc, X0, t, sim):
    h = t[1] - t[0]
    print(f"Integrating with h={h} (found h={sim.h} in sim)")
    res = [X0]
    for tv in t:
        # Do one Euler integration step for each time step...
        # https://personal.math.ubc.ca/~pwalls/math-python/differential-equations/first-order/
        X0 = X0 + h * dXfunc(X0, tv, sim)
        res.append(X0)
    return np.array(res[:-1])
    
def phase_coherence(angles_vec):
        '''
        Compute global order parameter R_t - mean length of resultant phase vector
        '''
        suma = sum([(np.e ** (1j * i)) for i in angles_vec])
        return abs(suma / len(angles_vec))


# REALTIME SIMULATION FUNCTIONS...

# Main simulation coroutine
async def sim_task(sim):
    """
    Entry point for Simulation process.
    """
    if sim.send_osc:
        if sim.osc_client is not None: # OSC client already exists
            #del sim.osc_client
            #sim.osc_client=None
            print("WARNING: OSC Client Already Exists, using pre-existing client.")
        else:
            sim.osc_client = SimpleUDPClient(sim.osc_dest_addr, sim.osc_dest_port)
        
        if sim.initial_volume is None:
            sim.initial_volume = 1.0
        sim.osc_client.send_message(sim.osc_msg_address, ["volume", sim.initial_volume]); #unmute sonification
    
    print("Starting Simulation...")
    sim.step=0

    # TODO: Add option to run for N steps
    while sim.run: # A single simulation Step
        sim.running = True
        timestamp = time()
        
        # Do one Euler step
        # https://personal.math.ubc.ca/~pwalls/math-python/differential-equations/first-order/
        sim.dX = sim.dXfunc(sim.X, sim.t, sim)
        sim.X = sim.X + sim.h * sim.dX
        sim.t += sim.h

        if sim.step % sim.timesteps_per_message == 0:
            
            osc_parcel = ["sim", sim.name, "t", sim.t]        
            osc_parcel.append("X"); osc_parcel.append(sim.X.tolist());
            osc_parcel.append("dX"); osc_parcel.append(sim.dX.tolist());
            
            if sim.send_osc:
                sim.osc_client.send_message(sim.osc_msg_address, osc_parcel)
                
            if sim.realtime_plot is not None:
                sim.realtime_plot.update([np.sin(sim.X).tolist()])
                # TODO: If the realtime plot is full, reset?

            if sim.print != False:
                if sim.step % sim.print_every == 0:
                    print(osc_parcel)
            
            await asyncio.sleep(sim.message_rate) # scheduler tick

        sim.t += sim.h
        sim.step += 1

    # Resource cleanup...
    print("Ending Simulation")
    if sim.osc_client is not None:
        sim.osc_client.send_message(sim.osc_msg_address, ["volume", 0.0]); #mute sonification
        del sim.osc_client
        sim.osc_client=None
    if sim.realtime_plot is not None:
        sim.realtime_plot.finalize()
        sim.realtime_plot = None
    
    sim.running = False

    
async def run_sim(sim, osc=True, print_output=False, print_every=1000):
    '''
    Run a real-time simulation.
    '''

    await stop_sim(sim)

    sim.run = True
    sim.send_osc = osc
    sim.print = print_output
    sim.print_every = print_every
            
    sim.X = sim.X0
    sim.t = 0
    
    try:
        loop=asyncio.get_event_loop()
        task=loop.create_task(sim_task(sim))
        print("Simulation Running...")
    except KeyboardInterrupt as e:
        exp = traceback.format_exc()
        raise e
    except Exception as e:
        exp = traceback.format_exc()
        print("Exit with Errors")
        raise e
    finally:
        # Do any necessary cleanup....
        print("Cleanup")

async def stop_sim(sim):
    if not hasattr(sim, "run"):
        sim.run = False

    if sim.run:
        # Wait for sim to finish
        sim.run = False
        # Stop any plots 
        if sim.realtime_plot is not None:
            sim.realtime_plot.finalize()
            sim.realtime_plot = None
        while sim.running:
            await asyncio.sleep(0.1)

            
async def plot_running_sim(sim, plt_height=500, plt_width=800):
    """
    Create a realtime plot for the currently running sim.    
    """
    if sim.realtime_plot is not None: 
        print("WARNING: A realtime plot already exists for this simulation!")
        sim.realtime_plot.finalize()
        sim.realtime_plot = None
        IPython.display.clear_output() 

    line_names = [str(x) for x in range(len(sim.X0))]
    sim.realtime_plot = ProgressPlot(line_names=line_names, y_lim=[-1.0, 1.0], width=plt_width, height=plt_height)
            
async def plot_with_refresh(sim, plt_height=500, plt_width=1000):
    sim.refresh_plot = True
    while sim.refresh_plot:
        await plot_running_sim(sim=sim, plt_height=plt_height, plt_width=plt_width)
        await asyncio.sleep(sim.refresh_plot_every)
        IPython.display.clear_output()
        if not sim.refresh_plot:
            sim.realtime_plot.finalize()
            sim.realtime_plot = None
            break

def plot_realtime(sim, auto_refresh=False, refresh_every=30, plt_height=500, plt_width=1000):
    sim.refresh_plot = auto_refresh
    sim.refresh_plot_every = refresh_every
    loop=asyncio.get_event_loop()
    if sim.refresh_plot:
        task=loop.create_task(plot_with_refresh(sim, plt_height=plt_height, plt_width=plt_width))
    else:
        task=loop.create_task(plot_running_sim(sim, plt_height=plt_height, plt_width=plt_width))


In [ ]:
# Start an OSCPipe to connect to SC
osc = oscpipe.BaseOSCPipe(
    logger=None,
    bind_ip=('localhost', 1337),
    client_ip=('localhost', 57120),
    osc_keepalive_threshold=30,
    outgoing_addr_prefix='/kuramoto',
)

aloop=asyncio.get_event_loop()
osc_server_task = aloop.create_task(osc.run_server(), name="OSC Server Task")
keepalive_osc_task = aloop.create_task(osc.run_osc_keepalive(), name="OSC Keepalive Task")


In [ ]:
# Some example messages for sanity...
# /python 'sim' simType 't' t_val 'X' [ X1 X2 X3 ... ] 'dX' [ dX1 dX2 dX3 ... ]
await osc.broadcast('sim', ['KuramotoBasic', 't', 0.0, 
                            'X', np.random.rand(20).tolist(), 
                            'dX',np.random.rand(20).tolist()]
                   )
await osc.broadcast('volume', [0.5])

In [ ]:
await osc.broadcast('volume', [0.0]) # Silence...

In [ ]:
# Kuramoto Simulator Implementation

if sim_task is not None:
    sim_task.cancel()

# Set up simulation state object
sim = type('KuramotoBasic', (), {})()
sim.name = "KuramotoBasic"
def set_connectivity(sim, connectivity=1.0):
    """Build the connectivity graph"""
    sim.connectivity=connectivity
    graph = nx.erdos_renyi_graph(n=sim.n, p=sim.connectivity)
    sim.connectivity_matrix = nx.to_numpy_array(graph) # connectivity matrix p=1. fully connected
    return graph

sim.set_connectivity = set_connectivity

def kuramoto_derivative(X, t, sim):
    '''
        Compute derivative (dtheta_i/dt) of all nodes at current state.
        Where:
            X: vector of current positions (angles theta) size N=number of nodes
            t: current time (this is an autonomous system, does not vary with t)
            sim: simulation state
    '''
    assert len(X) == len(sim.natural_frequencies) == len(sim.connectivity_matrix), 'Input dimensions do not match, check lengths'

    angles_i, angles_j = np.meshgrid(X, X)
    interactions = sim.connectivity_matrix * np.sin(angles_j - angles_i)  # Aij * sin(j-i)
    dXdt = sim.natural_frequencies + sim.k * interactions.sum(axis=0)  # sum over incoming interactions
    return dXdt

sim.dXfunc = kuramoto_derivative


# Important sim parameters
sim.n=20 # number of oscillators
sim.Xhist=np.atleast_2d(np.zeros(sim.n)).T # History of X values for each oscillator, used by plot
sim.k = 0.72    # global coupling constant K, typical values 0.1-2.0 ... usually phase shift at 0.15
sim.X0 = 2 * np.pi * np.random.random(size=sim.n) # initial positions theta
sim.natural_frequencies = np.random.normal(size=sim.n)  # normal distribution of natural frequencies omega
sim.h = 0.01 # integration time step
sim.t = 0.
sim.initial_volume = 0.5
sim.osc_message_rate = 0.1            # 20fps - delay between updates (OSC & graph) in seconds
sim.steps_per_osc_message = 10     # number of integration time steps calculated per update
sim.set_connectivity(sim, 1.) # p=1. = 100% fully connected network

# Momentary position & velocity
sim.X = sim.X0
sim.dX = np.array([])

# Run sim function...
async def run_sim(sim, num_steps=100, update_every=None):
    try:
        if update_every is not None:
            sim.osc_message_rate = update_every
        sim.step=0
        print(f"RUN SIM for {num_steps} steps, update every {sim.osc_message_rate}s, beginning with step {sim.step} ...")
        startTime = time.time()
        while sim.step < num_steps:

            # Do one Euler step
            sim.dX = sim.dXfunc(sim.X, sim.t, sim)
            sim.X = sim.X + sim.h * sim.dX
            sim.t += sim.h

            # Append sin(X) to Xhist
            sim.Xhist = np.append(sim.Xhist, np.atleast_2d(np.sin(sim.X)).T, 1)
            
            if sim.step % sim.steps_per_osc_message == 0:
                #print(f"simname:{sim.name} -- sim.t:{sim.t}")
                #print(f"Sim.dX: {sim.dX.shape} {sim.dX}")
                if osc is not None:
                    oscmsg = [sim.name, 't', sim.t, 
                        'X', sim.X.tolist(), 
                        'dX', sim.dX.tolist()
                    ]
                    await osc.broadcast('sim', oscmsg)   
                
                await asyncio.sleep(sim.osc_message_rate) # scheduler tick

            sim.step += 1
            
        executionTime = (time.time() - startTime)
        print(f"SIM END after {num_steps} steps and {str(executionTime)} seconds")  
    except Exception as ex:
        traceback.print_exc()
        print(f"Exception: {ex.__class__.__name__}: {str(ex)}")
 

In [ ]:
# Run sim
await osc.broadcast('volume', [0.5])

loop=asyncio.get_event_loop()
sim_task=loop.create_task(run_sim(sim, num_steps=100000))

In [ ]:
try:
    exception = sim_task.exception()
    if exception is not None:
        raise exception
except asyncio.InvalidStateError as e:
    sim_task.cancel()

sim_task.cancel()
await osc.broadcast('volume', [0.0])

In [ ]:
# Create a realtime plot
%matplotlib notebook

plot = type('AnimatedPlot', (), {})()
plot.name = 'AnimatedPlot'
plot.x_len = 1000         # Number of points to display
plot.y_range = [-1.01, 1.01]  # Range of possible Y values to display
plot.x_domain = range(0, 200)
plot.n = sim.n
plot.sim = sim # Sim to plot..

# Create figure, subplot and set y domain
plot.fig = plt.figure()
plot.fig.set_figwidth(9)
plot.fig.set_figheight(5)
plot.ax = plot.fig.add_subplot(1, 1, 1)
plot.ax.set_ylim(plot.y_range)
# Create a 0 line for each sim oscillaor, We will update the lines in animate
plot.lines = []
for i in range(plot.n):
    line, = plot.ax.plot(range(0, plot.x_len), np.zeros(plot.x_len))
    plot.lines.append(line)

# Add labels
plt.title('Data Realtime')
plt.xlabel('Samples')
plt.ylabel('Value')

# This function is called periodically from FuncAnimation
def plot_animate_frame(i, plot):
    #print(f"Animate!{i} {plot}")
    try:
        # Take the last x_len data points from the sim history
        ys = plot.sim.Xhist.T[-plot.x_len:].T
        padding = plot.x_len - ys.shape[1]
        if padding > 0:
            # Leading buffer with zeros
            ys = np.append(np.zeros([ys.shape[0], padding]), ys, 1)

        # Add y to each list
        for i in range(len(ys)):
            # Update each line
            plot.lines[i].set_ydata(ys[i])

        return plot.lines
    except Exception as ex:
        print(f"GOT ERROR IN ANIMATION LOOP: {ex.__class__.__name__}: {str(ex)}")

# Set up plot to call animate() function periodically
ani = animation.FuncAnimation(
    plot.fig,
    plot_animate_frame,
    fargs=(plot,),
    interval=100, # in ms
    blit=True
)

plt.show()

In [ ]:
# Manipulate some simulation parameters...
sim.k = 0.5
sim.set_connectivity(sim, 1)
sim.h = 0.005
#sim.natural_frequencies = np.random.normal(size=sim.n) * 6
sim.step = 0